In [ ]:
import efel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy
import funcs_for_ck2_sk_study as ephys
from scipy.signal import find_peaks
from scipy import signal
from scipy import stats
import seaborn as sns
import glob
import matplotlib
sns.set_style("ticks")
sns.set_context("paper")
plt.rcParams['font.family'] = 'Arial'
#plt.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['pdf.fonttype'] = 42

In [ ]:
#set a directory, all code for these analyses function for batch processing, and will work on a file's worth of data. They will specifically pull files in the axograph file format, and analyze them for each function. certain functions are dependent on other functions. 
#the purpose of this example notebook is for a general workflow for analyzing data. We have included a few functions and example data to show how to use the functions.
example_dir = os.getcwd()

In [ ]:
#Here, we will collect all interspike intervals and global mean traces for a set of files. This function will return a dataframe with the interspike intervals and global mean traces for each file, we will call these 'Control'. You can check the params for each function to see the input parameter requirements. 
control_noise_mean_trace, control_noise_isi, control_noise_global_mean_traces, control_noise_total_trajectories_baseline, control_noise_global_mean_isis = ephys.collect_isis_global_excel(example_dir, 20000, 0, 50, 0, 10, 'control')

In [ ]:
#Here we will plot the voltage trajectories for the control group. This function will plot the voltage trajectories for the control group. You can check the params for each function to see the input parameter requirements.
ephys.plot_average_voltage_trajectory(control_noise_mean_trace, control_noise_isi, 'Control', 'black', False, 'Control')

In [ ]:
#here we calculate the slopes and mean voltages for the group. This function will calculate the slopes and mean voltages for the group. You can check the params for each function to see the input parameter requirements.
#control slopes and mean voltages

save_path = 'Output path'
control_slopes = ephys.calculate_slope_and_mean_voltage_50ms_window(control_noise_global_mean_traces, control_noise_global_mean_isis, save_path, 'control')

control_slopes

In [ ]:
os.chdir('..')

In [ ]:
#Here we collect the spike voltages to work off of
control_global_spike_voltages, control_mean_spike_voltages = ephys.mean_spike_voltages(os.getcwd(), 0, 50, -10, 'Control')

In [ ]:
os.chdir('..')

In [ ]:
#calcualte the spike width and thresholds from the spike voltages
ephys.calculate_spike_widths_and_thresholds(control_global_spike_voltages, 20_000, 10, 'WT', save_path)

In [ ]:
#here we calculate the frequency and cv for the group. This function will calculate the frequency and cv for the group. You can check the params for each function to see the input parameter requirements.
ephys.collect_frequency_and_cv(os.getcwd(), 20000, 0, -10, 0, 10, 'control')